# Transformaciones
En este notebook se harán las transformaciones que se discuten en el notebook de exploración de datasets.

In [1]:
import pandas as pd

## Producto.parquet

In [2]:
prod = pd.read_parquet('../datasets/producto.parquet')
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [3]:
# ids unicos
prod.id.unique().shape

(72038,)

In [4]:
'''
# empezamos con los id
# son EAN, deberian ser de 13 digitos
# el otro formato de EAN que hallé (por ejemplo): 1-1-0000000205955
# nos quedamos con la ultima parte despues del segundo -
prod.id.loc[prod.id.str.len() != 13] = prod.id.loc[prod.id.str.len() != 13].str.split('-').str[2]


# chequeamos que se formatearon bien
prod.id.loc[prod.id.str.len() != 13]
''' 

"\n# empezamos con los id\n# son EAN, deberian ser de 13 digitos\n# el otro formato de EAN que hallé (por ejemplo): 1-1-0000000205955\n# nos quedamos con la ultima parte despues del segundo -\nprod.id.loc[prod.id.str.len() != 13] = prod.id.loc[prod.id.str.len() != 13].str.split('-').str[2]\n\n\n# chequeamos que se formatearon bien\nprod.id.loc[prod.id.str.len() != 13]\n"

In [5]:
# espacios en blanco y primera letra mayuscula
prod.marca = prod.marca.str.strip().str.capitalize()
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,La anónima,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,La anónima,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,Sin marca,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,Sin marca,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,Sin marca,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [6]:
# Eliminamos la presentacion del nombre
prod.nombre = prod.nombre.str.split('(\d+)', expand=True)[0]
# Espacios en blanco y letra mayuscula
prod.nombre = prod.nombre.str.strip().str.capitalize()
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,None,None,None
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,None,None,None
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,None,None,None
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,None,None,None
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,None,None,None


In [7]:
# dropeamos columnas inutiles
prod_aux = prod.loc[prod.categoria1.notnull()].copy()
prod_aux['tipoError'] = 0
prod_aux.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3,tipoError
14903,7501001165260,Pantene,Acondicionador liso y sedoso pantene,400.0 ml,Perfumería y Cuidado Personal,Cuidado Capilar,Acondicionadores,0
53619,7798024299045,None,None,None,Almacén,Aceites,Aerosol,0
55798,7798061190213,None,None,None,Almacén,Aceites,Oliva,0
65297,7798183771123,Grido,Helado en pote tentacion,1.0 lt,Alimentos Congelados,Helados,Helados,0


In [8]:
# columnas
prod.drop(['categoria1','categoria2','categoria3'],axis='columns', inplace=True)
prod.head()

# filas (coinciden con las filas que se copiaron a aux)
prod.drop(prod.loc[(prod.nombre.isnull()) | (prod.marca.isnull())].index, axis='index', inplace=True)

prod.head()

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg


In [9]:
# tipos de unidades que hay
prod.presentacion.str.split(expand=True)[1].unique()

# puedo disminuir la cantidad de unidades
# 1000 ml =  1000 cc = 1 lt
# 1000 gr = 1 kg 

array(['un', 'kg', 'gr', 'ml', 'cc', 'lt', 'mt'], dtype=object)

In [10]:
prod['cantidad'] = prod.presentacion.str.split(expand=True)[0].astype(float)
prod['unidad'] = prod.presentacion.str.split(expand=True)[1]
prod.head()

,id,marca,nombre,presentacion,cantidad,unidad
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,1.0,un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,1.0,un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,1.0,kg
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,1.0,kg
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,1.0,kg


In [11]:
prod.loc[prod.unidad == 'kg', 'cantidad'] = prod.cantidad * 1000
prod.loc[prod.unidad == 'kg', 'unidad'] = 'gr'

prod.loc[prod.unidad == 'lt', 'cantidad'] = prod.cantidad * 1000
prod.loc[prod.unidad == 'lt', 'unidad'] = 'ml'
prod.loc[prod.unidad == 'cc', 'unidad'] = 'ml'

# Tenemos menos unidades
prod.unidad.unique()

array(['un', 'gr', 'ml', 'mt'], dtype=object)

In [12]:
prod

,id,marca,nombre,presentacion,cantidad,unidad
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,1.0,un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,1.0,un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,1000.0,gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,1000.0,gr
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,1000.0,gr
...,...,...,...,...,...,...
72033,9569753142128,Deli-sitas,Milhojas cobertura de chocolate blanco deli-sitas,500.0 gr,500.0,gr
72034,9795403001143,Mayo,Mini pizzetas mayo,12.0 un,12.0,un
72035,9990385651922,Dana,Te negro en hebras lata dana,50.0 gr,50.0,gr
72036,9990385651939,Dana,Te verde en hebras lata dana,50.0 gr,50.0,gr


In [13]:
# juntamos cantidad y unidad y sustituimos presentacion
prod['presentacion'] = prod.cantidad.astype(str) + ' ' + prod.unidad
prod.drop(['cantidad','unidad'], axis='columns', inplace=True)
prod.head()

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1000.0 gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1000.0 gr
4,0000000205955,Sin marca,Chiquizuela novillito,1000.0 gr


In [14]:
# Para normalizar, hacemos una tabla de marcas
df_marca = prod.marca.to_frame().copy()
df_marca.drop_duplicates(inplace=True)
df_marca.insert(0, 'marcaId', range(1, 1 + len(df_marca)))
df_marca

,marcaId,marca
0,1,La anónima
2,2,Sin marca
24,3,Paladini
25,4,La paulina
26,5,Dulcor
...,...,...
70944,5561,La hacieda bandeja
71193,5562,Nexcare
72027,5563,Pez
72030,5564,Dilmah


In [15]:
# Mapping a producto
prod = pd.merge(prod, df_marca, on=['marca'])
prod.drop('marca', axis=True, inplace=True)
prod


,id,nombre,presentacion,marcaId
0,0000000001663,Radicheta atada la anonima,1.0 un,1
1,0000000002288,Perejil atado la anonima,1.0 un,1
2,2000957000007,Piña la anonima,1000.0 gr,1
3,2001271000001,Carre de cerdo con hueso la anonima,1000.0 gr,1
4,2001273000009,Pechito con hueso de cerdo la anonima,1000.0 gr,1
...,...,...,...,...
72031,9312631143560,Te en saquitos variety frutal dilmah,20.0 un,5564
72032,9312631144451,Te green en saquitos variety pack dilmah,20.0 un,5564
72033,9990385651922,Te negro en hebras lata dana,50.0 gr,5565
72034,9990385651939,Te verde en hebras lata dana,50.0 gr,5565


## sucursal.csv

In [16]:
suc = pd.read_csv('../datasets/sucursal.csv')
suc.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


In [17]:
# Tabla de comercio-bandera
df_bandera = suc[['banderaId','comercioId', 'banderaDescripcion', 'comercioRazonSocial']].copy()
df_bandera.drop_duplicates(inplace=True)
df_bandera.reset_index(inplace=True, drop=True)
# agrego un id
df_bandera.insert(0, 'comecioBanderaId', range(1, 1 + len(df_bandera)))
df_bandera.head()

,comecioBanderaId,banderaId,comercioId,banderaDescripcion,comercioRazonSocial
0,1,1,1,Super MAMI,Dinosaurio S.A.
1,2,1,10,Hipermercado Carrefour,INC S.A.
2,3,2,10,Market,INC S.A.
3,4,3,10,Express,INC S.A.
4,5,2,11,Changomas,Wal Mart Argentina S.R.L.


In [18]:
# Mapeamos el id a sucursal
suc = pd.merge(suc, df_bandera, on = ['banderaId','comercioId', 'banderaDescripcion','comercioRazonSocial'])
suc.drop(['banderaId','comercioId', 'banderaDescripcion', 'comercioRazonSocial'], axis='columns', inplace=True)
suc.head()


,id,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo,comecioBanderaId
0,1-1-7,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado,1
1,10-1-1,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado,2
2,10-1-10,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado,2
3,10-1-11,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado,2
4,10-1-112,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado,2


In [19]:
# Tabla tipo de sucursal
df_tipo_suc = suc['sucursalTipo'].copy()
df_tipo_suc.drop_duplicates(inplace=True)
df_tipo_suc = df_tipo_suc.to_frame()
df_tipo_suc.insert(0, 'tipoSucursalId', range(1, 1 + len(df_tipo_suc)))
df_tipo_suc.reset_index(inplace=True, drop=True)
df_tipo_suc

,tipoSucursalId,sucursalTipo
0,1,Hipermercado
1,2,Supermercado
2,3,Autoservicio
3,4,Minorista


In [20]:
# Mapeamos el id a sucursal
suc = pd.merge(suc, df_tipo_suc, on = ['sucursalTipo'])
suc.drop(['sucursalTipo'], axis='columns', inplace=True)
suc.head()

,id,provincia,localidad,direccion,lat,lng,sucursalNombre,comecioBanderaId,tipoSucursalId
0,1-1-7,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,1,1
1,10-1-1,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,2,1
2,10-1-10,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,2,1
3,10-1-11,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,2,1
4,10-1-112,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,2,1


In [21]:
# tabla de provincia y localidad
df_provincia = suc['provincia'].copy().to_frame()
df_provincia.drop_duplicates(inplace=True)
df_provincia.insert(0, 'provinciaId', range(1, 1 + len(df_provincia)))
df_provincia.reset_index(drop=True,inplace=True)

df_provincia

,provinciaId,provincia
0,1,AR-X
1,2,AR-B
2,3,AR-A
3,4,AR-U
4,5,AR-L
5,6,AR-R
6,7,AR-Q
7,8,AR-W
8,9,AR-V
9,10,AR-M


In [22]:
# tabla de localidad
df_localidad = suc[['provincia','localidad']].copy()
df_localidad.drop_duplicates(inplace=True)
# arreglamos strings
df_localidad.localidad = df_localidad.localidad.str.strip().str.capitalize()
# Mapeamos los id de provincia
df_localidad = pd.merge(df_localidad, df_provincia, on = ['provincia'])
df_localidad.drop(['provincia'], axis='columns', inplace=True)
# insetamos un index
df_localidad.insert(0, 'localidadId', range(1, 1 + len(df_localidad)))
df_localidad.reset_index(drop=True,inplace=True)

df_localidad


,localidadId,localidad,provinciaId
0,1,Salsipuedes,1
1,2,Jardin espinoza,1
2,3,Quintas de arguello,1
3,4,Villa allende lomas,1
4,5,Villa urquiza,1
...,...,...,...
686,687,Barrio rawson,24
687,688,San juan,24
688,689,Barrio rivadavia,24
689,690,San juan,24


In [23]:
# Mapeamos localidad a sucursal
df_localidad_map = pd.merge(df_localidad, df_provincia, on = ['provinciaId'])
suc = pd.merge(suc, df_localidad_map, on = ['localidad','provincia'])
suc.drop(['provincia','localidad','provinciaId'], axis='columns', inplace=True)
suc 

,id,direccion,lat,lng,sucursalNombre,comecioBanderaId,tipoSucursalId,localidadId
0,10-1-10,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,2,1,68
1,10-1-10,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,2,1,276
2,10-2-218,Av. Vergara 4121,-34.591972,-58.636447,Hurlingham,3,1,68
3,10-2-218,Av. Vergara 4121,-34.591972,-58.636447,Hurlingham,3,1,276
4,10-3-472,Jauretche 1125,-34.589617,-58.632800,Jauretche 1125,4,3,68
...,...,...,...,...,...,...,...,...
817,23-1-6284,Alvearn 831,-31.407600,-64.178600,Guzman,22,3,65
818,23-1-6286,Bulnes 1108,-31.399800,-64.175700,Bulnes,22,3,14
819,23-1-6286,Bulnes 1108,-31.399800,-64.175700,Bulnes,22,3,65
820,23-1-6265,Ruta 168. Km. 476 None,-31.641400,-60.659100,Santa Fe,22,3,552


In [24]:
# reordenamos
suc = suc[['id','sucursalNombre','comecioBanderaId','tipoSucursalId','localidadId', 'direccion','lat','lng']]
suc.head()

,id,sucursalNombre,comecioBanderaId,tipoSucursalId,localidadId,direccion,lat,lng
0,10-1-10,Villa Tesei,2,1,68,Av. Vergara 1910,-34.620610,-58.633769
1,10-1-10,Villa Tesei,2,1,276,Av. Vergara 1910,-34.620610,-58.633769
2,10-2-218,Hurlingham,3,1,68,Av. Vergara 4121,-34.591972,-58.636447
3,10-2-218,Hurlingham,3,1,276,Av. Vergara 4121,-34.591972,-58.636447
4,10-3-472,Jauretche 1125,4,3,68,Jauretche 1125,-34.589617,-58.632800


In [25]:
# strings de nombre y direccion
suc.sucursalNombre = suc.sucursalNombre.str.strip().str.capitalize()
suc.direccion = suc.direccion.str.strip().str.capitalize()

In [26]:
# renombramos columnas
suc.rename(columns={'id': "sucursalId", 'lat': 'latitud', 'lng': 'longitud'}, inplace=True)
suc

,sucursalId,sucursalNombre,comecioBanderaId,tipoSucursalId,localidadId,direccion,latitud,longitud
0,10-1-10,Villa tesei,2,1,68,Av. vergara 1910,-34.620610,-58.633769
1,10-1-10,Villa tesei,2,1,276,Av. vergara 1910,-34.620610,-58.633769
2,10-2-218,Hurlingham,3,1,68,Av. vergara 4121,-34.591972,-58.636447
3,10-2-218,Hurlingham,3,1,276,Av. vergara 4121,-34.591972,-58.636447
4,10-3-472,Jauretche 1125,4,3,68,Jauretche 1125,-34.589617,-58.632800
...,...,...,...,...,...,...,...,...
817,23-1-6284,Guzman,22,3,65,Alvearn 831,-31.407600,-64.178600
818,23-1-6286,Bulnes,22,3,14,Bulnes 1108,-31.399800,-64.175700
819,23-1-6286,Bulnes,22,3,65,Bulnes 1108,-31.399800,-64.175700
820,23-1-6265,Santa fe,22,3,552,Ruta 168. km. 476 none,-31.641400,-60.659100


## Precios
### precios_semana_20200413.csv

In [27]:
precios_0413 = pd.read_csv('../datasets/precios_semana_20200413.csv')
precios_0413.head()

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248
